In [22]:
print("kaggle 개인 토큰 파일 (kaggle. json)을 업로드 하셨나요?")

kaggle 개인 토큰 파일 (kaggle. json)을 업로드 하셨나요?


In [24]:
import os
import shutil

answer = input("kaggle 개인 토큰 파일 (kaggle.json)을 구글코랩에 업로드 하셨나요? (Y/N): ")
if answer.lower() == 'y':
    src_path = '/content/kaggle.json'
    dst_dir = '/root/.config/kaggle'
    os.makedirs(dst_dir, exist_ok=True)
    shutil.copy(src_path, os.path.join(dst_dir, 'kaggle.json'))
    print(f"kaggle.json 파일이 {dst_dir}로 복사되었습니다.")
else:
    print("먼저 kaggle.json 파일을 업로드해주세요.")

kaggle 개인 토큰 파일 (kaggle.json)을 업로드 하셨나요? (Y/N): Y
kaggle.json 파일이 /root/.config/kaggle로 복사되었습니다.


In [13]:
#데이터셋 불러오기
import os
import zipfile
!chmod 600 /root/.config/kaggle/kaggle.json


# 데이터셋 다운로드
!kaggle competitions download -c ai03-level1-project -p ./data



 99% 3.89G/3.91G [00:50<00:00, 102MB/s] 
100% 3.91G/3.91G [00:50<00:00, 83.0MB/s]


In [14]:
import zipfile
import os

zip_path = "./data/ai03-level1-project.zip"
extract_path = "./data/raw"

# 추출 경로가 없으면 생성
os.makedirs(extract_path, exist_ok=True)

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"'{zip_path}' 파일이 '{extract_path}' 경로에 성공적으로 압축 해제되었습니다.")
else:
    print(f"'{zip_path}' 파일을 찾을 수 없습니다. 다운로드가 완료되었는지 확인해주세요.")

'./data/ai03-level1-project.zip' 파일이 './data/raw' 경로에 성공적으로 압축 해제되었습니다.


# 데이터 개관/ 파일 구조 갯수

In [15]:
import os

# 압축 해제된 파일 목록 확인
extracted_files = os.listdir("./data/raw")
print("압축 해제된 파일 목록:")
for file_name in extracted_files:
    print(file_name)

압축 해제된 파일 목록:
train_annotations
train_images
test_images


In [25]:
import os

# train_annotations 폴더 안의 파일 목록 확인
annotation_files = os.listdir("./data/raw/train_annotations")
print("train_annotations 폴더의 디렉토리 목록:")
for file_name in annotation_files:
    print(file_name)

train_annotations 폴더의 디렉토리 목록:
K-003483-025367-027733-036637_json
K-003483-016262-027653-030308_json
K-003483-016232-025469-034597_json
K-003483-016232-025367-027777_json
K-003544-010221-016551-033009_json
K-002483-004378-019552-025438_json
K-003483-020238-027777-028763_json
K-003483-016262-022347-027733_json
K-003351-018357-036637_json
K-003351-018357-022074_json
K-003483-027653-030308-035206_json
K-003544-010221-016551-029345_json
K-002483-003743-013395-023223_json
K-003483-020877-022347-035206_json
K-003483-016262-027777-028763_json
K-001900-010224-016551-031705_json
K-003351-022074-033880_json
K-003483-027653-029667-030308_json
K-003483-016232-027733-030308_json
K-003351-016688-020014_json
K-002483-005886-012778-025438_json
K-003483-025469-028763-029667_json
K-003483-022347-027777-036637_json
K-003483-019861-025367-035206_json
K-003483-025367-027777-036637_json
K-003483-027733-028763-036637_json
K-001900-016551-029345-033208_json
K-003351-003832-029667_json
K-003483-027653-029667-0

In [18]:
import os

# 첫 번째 어노테이션 디렉토리 안의 파일 목록 확인
annotation_dirs = os.listdir("./data/raw/train_annotations")
if annotation_dirs:
    first_annotation_dir = annotation_dirs[0]
    dir_path = os.path.join("./data/raw/train_annotations", first_annotation_dir)

    if os.path.isdir(dir_path):
        files_in_dir = os.listdir(dir_path)
        print(f"'{first_annotation_dir}' 디렉토리의 파일 목록:")
        for file_name in files_in_dir:
            print(file_name)
    else:
        print(f"'{first_annotation_dir}'는 디렉토리가 아닙니다.")
else:
    print("train_annotations 폴더에 디렉토리가 없습니다.")

'K-003483-025367-027733-036637_json' 디렉토리의 파일 목록:
K-025367
K-036637
K-003483
K-027733


In [43]:
import os
from collections import defaultdict

base_dir = "./data/raw/train_annotations"
output_path = "/content/filestructure.txt"

# JSON 파일 목록 및 최심화 정보
json_files = []
max_depth = 0
deepest_json_path = None

# 폴더별 예시 파일과 파일 개수 저장
folder_examples = {}

# 깊이별 폴더/파일/하위폴더 개수 저장
depth_stats = defaultdict(lambda: {"folders": 0, "files": 0, "subfolders": 0})

for root, dirs, files in os.walk(base_dir):
    rel = os.path.relpath(root, base_dir)
    depth = 0 if rel == "." else len(rel.split(os.sep))

    # 1층부터 통계 집계
    if depth > 0:
        depth_stats[depth]["folders"] += 1
        depth_stats[depth]["files"] += len(files)
        depth_stats[depth]["subfolders"] += len(dirs)
        if files:
            folder_examples[rel] = (files[0], len(files))

    # JSON 파일 검색 및 최심화 업데이트
    for fname in files:
        if fname.lower().endswith(".json"):
            full_path = os.path.join(root, fname)
            json_files.append(full_path)
            if depth > max_depth:
                max_depth = depth
                deepest_json_path = full_path

# 폴더별 예시 파일 및 개수를 텍스트 파일로 저장
with open(output_path, "w", encoding="utf-8") as f:
    for folder, (fname, cnt) in sorted(folder_examples.items()):
        f.write(f"{folder} → {fname} ({cnt}개)\n")


if json_files:
    print("발견된 JSON 파일: (목록이 너무 길어 주석처리하고 txt로 저장함)")
  #  for p in json_files:
  #         print(f" - {p}")
    print(f"가장 깊은 위치의 JSON 파일은:\n  {deepest_json_path}")
else:
    print("찾을 수 있는 JSON 파일이 없습니다.")

# 출력: 폴더별 예시 파일 및 개수 (최초 5개만)
print("\n=== 폴더별 예시 파일 및 파일 개수 (최초 5개만) ===")
for i, (folder, (fname, cnt)) in enumerate(sorted(folder_examples.items())):
    if i >= 5:
        break
    print(f"{folder} → {fname} ({cnt}개)")

# 출력: 층별 폴더/파일/하위폴더 개수 및 평균
print("\n=== 층별 폴더/파일 개수 및 평균 ===")
for level in sorted(depth_stats):
    stats = depth_stats[level]
    avg_sub = stats["subfolders"] / stats["folders"] if stats["folders"] else 0
    avg_file = stats["files"] / stats["folders"] if stats["folders"] else 0
    print(
        f"{level}층: 폴더 {stats['folders']}개, "
        f"파일 {stats['files']}개, "
        f"하위폴더 합계 {stats['subfolders']}개  "
        f"(폴더당 평균 하위폴더 {avg_sub:.2f}개, 평균 파일 {avg_file:.2f}개)"
    )


print(f"예시 정보를 '{output_path}'에 저장했습니다.")


발견된 JSON 파일: (목록이 너무 길어 주석처리하고 txt로 저장함)
가장 깊은 위치의 JSON 파일은:
  ./data/raw/train_annotations/K-003483-025367-027733-036637_json/K-025367/K-003483-025367-027733-036637_0_2_0_2_75_000_200.json

=== 폴더별 예시 파일 및 파일 개수 (최초 5개만) ===
K-001900-010224-016551-031705_json/K-001900 → K-001900-010224-016551-031705_0_2_0_2_75_000_200.json (3개)
K-001900-010224-016551-031705_json/K-016551 → K-001900-010224-016551-031705_0_2_0_2_75_000_200.json (3개)
K-001900-010224-016551-031705_json/K-031705 → K-001900-010224-016551-031705_0_2_0_2_90_000_200.json (2개)
K-001900-010224-016551-033009_json/K-001900 → K-001900-010224-016551-033009_0_2_0_2_70_000_200.json (3개)
K-001900-010224-016551-033009_json/K-016551 → K-001900-010224-016551-033009_0_2_0_2_90_000_200.json (2개)

=== 층별 폴더/파일 개수 및 평균 ===
1층: 폴더 498개, 파일 0개, 하위폴더 합계 1876개  (폴더당 평균 하위폴더 3.77개, 평균 파일 0.00개)
2층: 폴더 1876개, 파일 4526개, 하위폴더 합계 0개  (폴더당 평균 하위폴더 0.00개, 평균 파일 2.41개)
예시 정보를 '/content/filestructure.txt'에 저장했습니다.


# 데이터 TEST와 Validation 나누기